# Eliminación por Bloques con el apoyo el método matricial de factorización QR 

## Introducción

Visualmente, la solución de sistemas de ecuaciones lineales (SEL) implica encontrar la intersección de rectas, planos o hiperplanos (dependiendo de las dimensiones en las que nos encontremos). Para la solución de este tipo de sistemas, existen diversos métodos, que son escogidos de conformidad con las características del sistema objeto de resolución (en específico, sus coeficientes).

Un forma de resolver estos SEL consiste en la eliminación de un subconjunto de variables y resolver un sistema más pequeño (`eliminación por bloques`), para ello, en el presente trabajo se implementó un algoritmo directo basado en factorizaciones matriciales (FM): `factorización QR` que permiten encuentrar sistemas de ecuaciones equivalentes (es decir, sistemas de ecuaciones lineales que tengan un mismo conjunto de solución). Una vez calculada la FM de la matriz a resolver, se utilizaron métodos de sustitución hacia adelante y hacia atrás para resolver SEL triangulares.



Para la implementación del presente trabajo, el equipo de programación desarrollo una serie funciones que permiten la implementación del algoritmo antes descrito; dichas funciones se encuentran principalmente en los siguientes el siguiente módulo:

> [funciones_factorizacion_QR.py](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Codigo/funciones_factorizacion_QR.py)

La descripción explícita de las funciones implementadas, los parámetros, resultados esperados y ejemplos de implementación de las funciones, se encuentran en el siguiente documento auxiliar:
> [Help_funciones_factorizacion_QR.txt](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Codigo/Help_funciones_factorizacion_QR.txt)

Durante la implementación de los métodos, gracias a los comentarios y revisiones del equipo revisor, se incluyeron las validaciones correspondientes sobre las condiciones necesarias para la aplicación del método de eliminación por bloques a la matriz $A$: matrix cuadrada ($nxn$) no singular. En caso de que no se cumplieran estas condiciones, el programa implementado arroja mensajes de error, como es esperado. Asimismo, se realizaron las mediciones de tiempo y errores relativos correspondientes para evaluar el desempeño del programa, ya sea por subdivisión de la matriz del sistema o no. 

El objeto del presente notebook es justamente reportar los hallazgos que obtuvimos durante la implementación del programa, ya sea durante de la programación del código o durante la revisión del algoritmo propuesto. Los detalles relacionados a la implementación del framwork de $scrum$ o la organización del trabajo son cubiertos en el [README.md](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/README.md) del repositorio.

## Sobre los métodos empleados

> Algoritmos

<p style='text-align: justify;'> Existen una gran cantidad de métodos para resolver SEL y por lo general se elige el método de acuerdo con los coeficientes de la matriz del sistema que determinan propiedades de la misma y sus dimensiones. Por lo que, para SEL no triangulares, más generales, es decir, que no cuentan con una estructura identificable, se tienen los métodos iterativos y directos o basados en factorizaciones matriciales (FM). Entre los directos o basados en FM se encuentra la factorización $QR$. 
    
La factorización QR se calculó con reflexiones de Householder, la cual consiste en utilizar matrices simétricas, ortogonales. La factorización $QR$ con refleciones de Householder se puede construir a partir de un vector ($v$), donde $v\in \mathbb{R}^{m}$ y debe ser diferente de cero. Una matrix $R$ de $m$x$m$ de la forma: $$R=I-\beta v v^{T}, \ \ \ \beta=\frac{{2}}{v^{T}v},$$ es una reflexión Householder. El vector $v$ es el vector Householder. Si un vector $x$ se multiplica por una matrix $R$, esto representa la reflexión del vector $x \in \mathbb{R}^{m}$ a través del hiperplano $span\{v\} ^{\perp}$. 

Las reflecciones de Householder son similares a las transformaciones de Gauss, ya que son modificaciones en el rango 1 de la identidad, en particular suponemos que $0 \neq x \in \mathbb{R}^{m}$ y queremos: $$Rx=\left(I - \frac{2vv^{T}}{v^{T}v}\right)x = x - \frac{2v^{T}x}{v^{T}v}v$$ debe ser un multiplo de $e_1=I_m(:,1)$. De esto concluimos que $v \in span\{x,e_1\}$. Ajustando: $$v = x + \alpha e_1$$ da $$v^{T}x = x^{T}x + \alpha x_1$$ y $$v^{T}v = x^{T}.$$ Por tanto, $$Rx = \left(1-2\frac{x^Tx+\alpha x_1}{x^Tx+2\alpha x_1 + \alpha_2}\right)x - 2 \alpha \frac{v^Tx}{v^Tv}e_1.$$ Para que el coeficiente de $x = 0$, ajustamos $\alpha = \pm ||x||_2$, para ello: $$v = x \pm ||x||_2 e_1 \Rightarrow Rx=\left(I - \frac{2vv^{T}}{v^{T}v}\right)x = \mp ||x||_2 e_1.$$ Es esta simple determinación de $v$ que hace que las reflexiones de Householder sean tan útiles.
    




</p>

## Funciones y módulos creados

Todas las función creadas para el proyecto se encuentran dentro del módulo *funciones_factorizacion_QR*. A continuación se enlistan y se da una breve descripción de las funcioines creadas para este proyecto:

- *crear_matriz_aleatoria*: crea matrices aleatorias. Devuelve una matriz $A$ con con los renglones y columnas deseados.
- *house*: calcula el vector de householder para un vector $\vec{v}$ dado.
- *matriz_auxiliar_Arv*: dada una matriz $A$, genera una matriz auxiliar que contiene los elementos $r$ distintos de cero de la matriz $R$ y las entradas de los vectores householder $\vec{v}$ (excepto la primera entrada), con los cuales se puede calcular la matriz $Q$ correspondiente.
- *Q_j*: calcula la j-esima iteración del proceso para la factorización $QR$, regresando la matriz $Q_j$ correspondiente con ayuda de la *matriz_auxiliar_Arv*.
- *matriz_Q_R*: dada una matriz $A$, devuelve la matrices $Q$ y $R$ de la factorización $QR$ apoyándose de la función *Q_j* mediante la aplicación del método iterativo, en el que $Q=Q_1Q_2\ldots Q_n$
- *Solucion_SEL_QR_nxn*: obtiene la solución de un sistema de ecuaciones lineales $Ax=b$ con $n$ ecuaciones y $n$ incognitas usando la factorización $QR$ y usando el método de sustitución hacia adelante del paquete *scipy*.
- *crear_bloques*: dada una matriz $A$, devuelve cuatro bloques que componen a esta matriz, en caso de no dar los argumentos para el tamaño de los bloques devuelve los bloques del mismo tamaño.
- *eliminacion_bloques*: implementa el método de eliminación por bloques para resolver un sistemade ecuaciones usando la factorización $QR$.

El módulo *funciones_factorización_QR* se encuentra en la siguiente liga [funciones](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Codigo/funciones_factorizacion_QR.py)

## Ejemplos de Aplicación

Con la intención de mostrar la funcionalidad de los algoritmos implementados por el equipo de programación, el equipo de revisión llevó a cabo una serie de mediciones clasificadas en los siguiente tres rubros:
+ Medición de tiempos a nivel granular
+ Medición de tiempos y errores a nivel general
+ Medición de tiempos y errores variando el tamaño de bloques

El desarrollo explícito de estas mediciones, puede encontrarse en la siguiente liga [insumos](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/tree/master/Reporte/Insumos).

En seguida se presentarán los resultados relacionados con cada una de las mediciones.


>Medición de tiempos a nivel granular

La función general de este proyecto, **eliminacion_bloques** utiliza a su vez un conjunto de funciones para resolver los sistemas de ecuaciones lineales. Estas funciones de apoyo pueden enunciarse particularmente conforme a lo siguiente:

1. matriz_auxiliar_Arv
2. matriz_Q_R (se hace uso de la función matri_auxiliar_Arv)
3. crear_bloques
4. Solucion_SEL_QR_nxn (se hace uso de la función matriz_Q_R)

Debido a la presencia de estas cuatro ecuaciones, se decidió medir sus tiempos de ejecución para distintos tamaños de matrices. Adicionalmente se incluyó también la medición de la matriz que permite crear matrices aleatorias (*crear_matriz_aleatoria*), si bien esta función no forma parte de la función general *eliminacion_bloques*, su uso es determinante para la creación de los sistemas de ecuaciones.

Es importante mencionar que la función *Solucion_SEL_QR_nxn* es utilizada cuatro veces en el algoritmo *eliminacion_bloques*; sin embargo, para este caso se midieron únicamente los tiempos asociados a la solución del sistema de ecuaciones correspondiente al primer bloque, es decir, $A_{11}y = b_1$

La medición de tiempos de ejecución se llevó a cabo sobre cuatro matrices de dimensiones $20x20$, $10x10$, $100x100$ y $500x500$; y los detalles relacionados con las mismas se muestran en [Tiempos_granular](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-qr-dapivei/blob/master/Reporte/Insumos/Tiempos_granular.ipynb). 

Como resumen de los resultados obtenidos, destacan los siguientes puntos:
+ En general, los tiempos de ejecución de las funciones aumentan conforme aumentan las dimensiones de la matriz. Excepto en la función *crear_bloques*, la cual mantiene un tiempo constante en la creación de submatrices para las 4 matrices que fueron probadas.
+ El hecho de que la función *crear_bloques* mantenga un tiempo constante puede deberse a que no se implementa ningún ciclo for, a diferencia del resto de las funciones, en donde al menos un ciclo es incluido.
+ La función matriz_QR es la que tiene asociada los tiempos más altos de ejecución, alcanzando los $3s$ en matrices de tamaño $500x500$.
+ La función *matriz_auxiliar_Arv* y *Solucion_SEL_QR* son las que presentan menores tiempos de ejecución, variando entre $10^{-4} s$ y $10^{-1} s$.

>Medición de tiempos y errores a nivel general

>Medición de tiempos y errores variando el tamaño de bloques

## Pruebas

>Factorización QR

>Sistemas con solución única

>Sistemas con infinitas soluciones

>Sistemas sin solución

## Conclusiones

## Referencias